In [ ]:
import numpy as np
from lightfm.datasets import fetch_movielens
from lightfm import LightFM
from lightfm.evaluation import precision_at_k
# https://grouplens.org/datasets/movielens/100k/
data = fetch_movielens(data_home='./data', min_rating=5.0)  #  获取数据集

print(repr(data['train']))  # 数据集中包含训练集和测试集 # coo_matrix 943
print(repr(data['test']))  # coo_matrix 943

'''
使用WARP（加权近似秩对）模型。
WARP是一个隐式反馈模型：训练矩阵中的所有交互都被视为正向，用户未与之隐式交互的产品则不喜欢。
该模型的目标是对这些隐含的正向评分较高，同时将较低的分数与隐含的负向评分相关联。
'''
model = LightFM(loss='warp')
%time model.fit(data['train'], epochs=30, num_threads=2)  # 时间

# 评估模型效果
# Precision @ k表示用户实际与之交互的排名中前k个项目的百分比
print("Train precision: %.2f" % precision_at_k(model, data['train'], k=5).mean())
print("Test precision: %.2f" % precision_at_k(model, data['test'], k=5).mean())


def sample_recommendation(model, data, user_ids):
    '''
    预测并推荐用户最感兴趣的产品
    '''
    n_users, n_items = data['train'].shape
    for user_id in user_ids:
        known_positives = data['item_labels'][data['train'].tocsr()[user_id].indices]
        scores = model.predict(user_id, np.arange(n_items))
        top_items = data['item_labels'][np.argsort(-scores)]
        print("User %s" % user_id)
        print("     Known positives:")
        for x in known_positives[:3]:
            print("        %s" % x)
        print("     Recommended:")
        for x in top_items[:3]:
            print("        %s" % x)

sample_recommendation(model, data, [3, 25, 450])